In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.cluster import KMeans
from sklearn import datasets
import random
import time
import pylab as pl
from sklearn import metrics, ensemble
import xgboost as xgb
import seaborn as sns
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import scipy.sparse as sp
import tensorflow as tf
from tqdm import tqdm

In [15]:
def init_variable(size, dim, name=None):
    '''
    Helper function to initialize a new variable with
    uniform random values.
    '''
    std = np.sqrt(2 / dim)
    return tf.Variable(tf.random.uniform([size, dim], -std, std), name=name)


def embed(inputs, size, dim, name=None):
    '''
    Helper function to get a Tensorflow variable and create
    an embedding lookup to map our user and item
    indices to vectors.
    '''
    emb = init_variable(size, dim, name)
    return tf.nn.embedding_lookup(emb, inputs)


def get_variable(graph, session, name):
    '''
    Helper function to get the value of a
    Tensorflow variable by name.
    '''
    v = graph.get_operation_by_name(name)
    v = v.values()[0]
    v = v.eval(session=session)
    return v

def find_similar_artists(artist=None, num_items=10):
    """Find artists similar to an artist.
    Args:
        artist (str): The name of the artist we want to find similar artists for
        num_items (int): How many similar artists we want to return.
    Returns:
        similar (pandas.DataFrame): DataFrame with num_items artist names and scores
    """

    # Grab our User matrix U
    user_vecs = get_variable(graph, sess, 'user_factors')

    # Grab our Item matrix V
    item_vecs = get_variable(graph, sess, 'item_factors')

    # Grab our item bias
    item_bi = get_variable(graph, sess, 'item_bias').reshape(-1)

    # Get the item id for Lady GaGa
    item_id = int(item_lookup[item_lookup.artist_name == artist]['artist_id'])

    # Get the item vector for our item_id and transpose it.
    item_vec = item_vecs[item_id].T

    # Calculate the similarity between Lady GaGa and all other artists
    # by multiplying the item vector with our item_matrix
    scores = np.add(item_vecs.dot(item_vec), item_bi).reshape(1,-1)[0]

    # Get the indices for the top 10 scores
    top_10 = np.argsort(scores)[::-1][:num_items]

    # We then use our lookup table to grab the names of these indices
    # and add it along with its score to a pandas dataframe.
    artists, artist_scores = [], []
    
    for idx in top_10:
        artists.append(item_lookup.artist_name.loc[item_lookup.artist_id == str(idx)].iloc[0])
        artist_scores.append(scores[idx])

    similar = pd.DataFrame({'artist': artists, 'score': artist_scores})

    return similar
